# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    body: str
    links: List[str]

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/08/06/outsmart/
https://edwarddonner.com/2024/08/06/outsmart/
https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/
https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/
https

In [9]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'news page', 'url': 'https://anthropic.com/news'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
New
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Get started now
Our Work
See All
Announcements
Claude 3.5 Sonnet
Jun 21, 2024
Alignment
·
Research
Constitutional AI: Harmlessness from AI Feedback
Dec 15, 2022
Announcements
Core Views on AI Safety: When, Why, What, and How
Mar 8, 2023
Work with Anthropic
Anthropic is an AI safety and research company based in San Francisco. Ou

In [22]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}]}


# Anthropic Brochure

---

## Welcome to Anthropic

At **Anthropic**, we are at the forefront of AI safety and research, dedicated to creating reliable, interpretable, and steerable AI systems that prioritize safety. Based in **San Francisco**, our interdisciplinary team—comprising researchers, engineers, policy experts, and business leaders—strives to ensure transformative AI benefits people and society as a whole.

---

## Our Mission

### **Building Safer AI**
- We believe in the vast potential of AI and are committed to building systems that can be reliably used in real-world applications.  
- Our research explores key areas such as interpretability, reinforcement learning from human feedback, and the societal impacts of AI.

### **AI Safety as a Science**
- Safety isn't just a goal; it's a systematic science. Our approach integrates rigorous research, product application, and continuous feedback to create safer AI models.

### **Collaborative Ecosystem**
- We are one piece of the larger AI landscape and actively collaborate with civil society, government, academia, and industry to promote wide-ranging safety measures in AI.

---

## Meet Claude

Introducing **Claude**, our cutting-edge AI model currently at version **3.5 Sonnet**. Claude is designed to assist in boosting productivity and creativity in various sectors. Some functionalities include:
- Drafting and debugging code
- Generating insightful reports and documents
- Assisting in marketing campaign strategies

### **What Customers Say**
> “Claude has transformed our workflows, allowing us to accomplish tasks up to **5x faster**!” - Luka Anic, Senior Director, North Highland

---

## Company Culture

**Values that Guide Us**
- **Mission-Driven:** Focused on ensuring that AI benefits society.
- **Trust:** We cultivate an open, high-trust environment that allows for honest communication and collaboration.
- **Teamwork:** Emphasizing collaboration across teams to harness diverse talents and ideas.
- **Pragmatism:** We value straightforward solutions that effectively balance trade-offs.

---

## Careers at Anthropic

Join our innovative team and make a difference in the AI landscape. We offer:
- **Competitive Compensation**: Salaries that reflect your expertise, with significant equity options.
- **Health & Wellness**: Comprehensive insurance benefits, 22 weeks of parental leave, and unlimited PTO.
- **Flexible Work Environment**: Opportunities for hybrid work and relocation support.

### **How We Hire**
Our interview process is designed to identify the best talent while minimizing bias. With multiple stages, including exploratory chats and technical assessments, we want to understand not only your skills but also how you align with our mission.

---

## Join Us!

Are you ready to make an impact in the field of AI? Visit our [Careers page](https://www.anthropic.com/careers) to explore current openings and join our mission to develop safer AI solutions for everyone.

---

For more information about Anthropic and our offerings, visit our [Website](https://www.anthropic.com).

**Follow Us:**
- [Twitter](https://twitter.com/anthropic)
- [LinkedIn](https://www.linkedin.com/company/anthropic)

---

© 2024 Anthropic PBC | All Rights Reserved  
**Privacy Policy** | **Terms of Service**  


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic Brochure

**Company Overview**  
Anthropic is a pioneering AI safety and research company based in San Francisco, dedicated to building reliable, interpretable, and steerable AI systems. Our mission is to ensure that transformative AI technologies help society flourish, navigating the complexities and risks associated with the advancement of AI.

**Meet Claude**  
Our flagship product, Claude, is an advanced AI model designed to assist teams and enhance productivity by streamlining workflows and facilitating collaboration. The latest version, Claude 3.5 Sonnet, exemplifies our commitment to safety and performance in AI.

![Claude](link-to-image)

## Our Vision
At Anthropic, we believe that AI will significantly affect the world. Our aim is to build systems that users can depend on while conducting research into the opportunities and risks AI presents. Our approach to safety is systematic and scientific, integrating research insights into our products while sharing our findings with the wider community.

## Company Culture
- **Interdisciplinary Collaboration**: Our team comprises experts from various fields, including machine learning, physics, policy, and business. We foster a collaborative environment where diverse perspectives help shape our projects.
- **High Trust Environment**: We promote an atmosphere of honesty, emotional maturity, and intellectual openness. This trust enhances our decision-making and strengthens our team dynamics.
- **Mission-Driven Approach**: All team members are aligned with our mission to make AI safer and beneficial for society. We believe in pragmatism, embracing simple yet effective solutions and continuous learning through empirical evidence.

## Customer Focus
We serve businesses, non-profit organizations, and civil society groups by translating our research into practical tools that facilitate enhanced workflows and better decisions across sectors. Notable sectors benefiting from Claude include engineering, marketing, sales, and customer support, generating measurable improvements in task efficiency.

## Join Our Team
We seek motivated individuals who are passionate about AI and its societal impacts. At Anthropic, employees enjoy several benefits, including:
- Competitive salary & equity packages.
- Comprehensive health, dental, and vision insurance.
- Unlimited paid time off (PTO).
- Support for ongoing education and home office improvement.
- A collaborative and inclusive workplace culture.

### Current Openings
If you are ready to make an impact in AI safety, explore our open roles on our [Careers Page](link-to-careers).

## Get in Touch
To learn more about Anthropic and our offerings, visit our website or follow us on our social media channels. We are committed to transparency and eagerly welcome inquiries from interested parties.

- **Website**: [Anthropic](https://www.anthropic.com)
- **Twitter**: [@Anthropic](https://twitter.com/anthropic)
- **LinkedIn**: [Anthropic](https://www.linkedin.com/company/anthropic)

### Join Us in Shaping the Future of AI
Let’s collaborate on building a safer, brighter future with AI technologies. 

---

*For press inquiries, please contact: press@anthropic.com*  
*For support and assistance, reach out to: support@anthropic.com* 

### Anthropic PBC © 2024


---

*This brochure provides a snapshot of Anthropic's mission, culture, and contributions to safer AI technologies. We invite you to explore partnership opportunities and join our dynamic team.*

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face!

## The Place Where AI Gets *Hugged*!

👋 Hey there, future innovators and trailblazers! Do you want to join a community dedicated to building the future of AI? Then look no further! Welcome to Hugging Face, the vibrant platform where the machine-learning community collaborates on models, datasets, and tons of applications. We promise we won't smother you with hugs... unless you want that!

---

### **About Us:**

At Hugging Face, we're on a heartwarming mission to democratize *good* machine learning, one commit at a time. That’s right, we’re making ML so accessible that even your grandma could model a neural network (with some help, of course). Our fantastic 217-member strong team of AI aficionados is unwaveringly dedicated to making ML fun and fruitful. We even have a secret handshake - just kidding, we don't have secrets or handshakes; we just have **open-source**!

---

### **What We Offer:**

- **Community Collaboration**: Share, discover, and collaborate on **400,000+** ML models and **100,000+** datasets! Our users have gone wild with creativity — from *text generation* to *audio and image solutions!*
- **Spaces for Everyone**: Like a high-tech playground, our Spaces allow you to run applications like *Kolors Virtual Try-On* and even generate *text-to-video*! Say cheese, please!
- **AI Tools That Hug You Back**: Dive right into our **HuggingChat**, where you can find vast AI tools available at your fingertips. And don’t worry, they come without awkward small talk!

---

### **Our Customers:**

With more than **50,000 organizations** using our platform, it seems we’re popular! Our clientele includes tech titans like:
- AI at Meta
- Amazon Web Services
- Google
- Intel
- Microsoft
- Grammarly

If you’re looking to join an entourage of heavyweight entities, you’ve landed in the right hug!

---

### **Careers:**

🎉 **Join the Hugging Face Family!** 🎉
Our door is always open for creative minds who want to sprinkle a little magic into the world of AI. We're on the hunt for talent in various areas. Think you can help us? Here's what you can expect:
- A team that celebrates diverse perspectives — we embrace differences like we embrace our cats!
- A chance to build your ML portfolio & get recognized (we mean MORE than just a thumbs-up emoji!).
- A flexible work environment where you can wear slippers to meetings if you choose (we won’t judge).

---

### **Closing Thoughts:**

So there you have it! At Hugging Face, we’re committed to building a friendly, collaborative, and innovative environment. Whether you're a customer, investor, or a potential recruit, remember: **We believe in hugging, not hacking!**

> So what are you waiting for? Join us, and let's build the future together! 🚀💚

[Visit Hugging Face](https://huggingface.co) to become part of our connected community! 